In [ ]:
import sys
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import timedelta
import time
import redis
import struct
from deepface import DeepFace
from deepface.basemodels import Facenet
from deepface.detectors import FaceDetector
from deepface.commons import functions

In [ ]:
!pip list

In [ ]:
r = redis.StrictRedis(host="redis",port= 6379,db=0)
r.ping()

In [ ]:
def toRedis(r,a,n):
   """Store given Numpy array 'a' in Redis under key 'n'"""
   h, w = a.shape
   shape = struct.pack('>II',h,w)
   encoded = shape + a.tobytes()

   # Store encoded data in Redis
   r.set(n,encoded)
   return

def fromRedis(r,n):
   """Retrieve Numpy array from Redis key 'n'"""
   encoded = r.get(n)
   h, w = struct.unpack('>II',encoded[:8])
   # Add slicing here, or else the array would differ from the original
   a = np.frombuffer(encoded[8:],dtype=np.uint16).reshape(h,w)
   return a

# Create 80x80 numpy array to store
#a0 = np.arange(6400,dtype=np.uint16).reshape(80,80) 

# Redis connection
#r = redis.Redis(host='localhost', port=6379, db=0)

# Store array a0 in Redis under name 'a0array'
#toRedis(r,a0,'a0array')

# Retrieve from Redis
#a1 = fromRedis(r,'a0array')

#np.testing.assert_array_equal(a0,a1 )

In [ ]:
video_file = '/opt/workspace/facial_database/datasets/videos/dream_scene.mp4'

In [2]:
one_person = '/opt/workspace/facial_database/datasets/actor_faces/2037_cillian_murphy/cillian_murphy_1.jpg'
two_person = '/opt/workspace/facial_database/datasets/two_people.jpeg'

### Face detection

In [ ]:
backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface']

#face detection and alignment
detected_face = DeepFace.detectFace(test_image, target_size = (160, 160) ,detector_backend = backends[0])

##face verification
#obj = DeepFace.verify("img1.jpg", "img2.jpg", detector_backend = backends[4])

#face recognition
#df = DeepFace.find(img_path = "img.jpg", db_path = "my_db", detector_backend = backends[4])

#facial analysis
#demography = DeepFace.analyze("img4.jpg", detector_backend = backends[4])

In [46]:
img = cv2.imread(two_person)
face = get_faces_from_img(img)

In [47]:
len(face)

2

In [45]:
def get_faces_from_img(img,detector_name = 'opencv'):
    detector = FaceDetector.build_model(detector_name)
    obj = FaceDetector.detect_faces(detector, detector_name, img)
    return obj

In [ ]:
#detect and align
test_image_3 = cv2.imread(test_image)
array = get_embedding_for_img(test_image_3)

In [131]:
def get_embedding_for_img(img,model = 'Facenet',size = (160,160)):
    model = DeepFace.build_model(model)
    try:
        proccesed_image = functions.preprocess_face(img, target_size = size)
        embedding = model.predict(proccesed_image)[0].tolist()
        arr = np.array(embedding)
        return arr        

In [ ]:
#store in redis
r.rpush("embedding:"+'Cillian', *embedding) #Pushing list to redis
r.set("photo:"+'Cillian', test_image)

In [ ]:
r.get("photo:Cillian")

In [ ]:
array = r.lrange("embedding:Cillian",0,-1)

In [ ]:
r.keys()

In [ ]:
array

In [ ]:
r.set("Hello","World")

In [ ]:
cap = cv2.VideoCapture(video_file)
video_fps = cap.get(cv2.CAP_PROP_FPS)
desired_fps = 1
amount_of_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
results = []
frame_no = 0

starttime = time.time()
while(cap.isOpened()):
    frame_exists, curr_frame = cap.read()
    if frame_exists:
        if frame_no % (round(video_fps/desired_fps)) == 0:
            #faces = get_faces_from_img(curr_frame)
            try:
                first_face_embedding = get_embedding_for_img(curr_frame)
                results.append(first_face_embedding)
            except:
                print("An error ocurred:",sys.exc_info()[0])
            

            #fix_frame = cv2.cvtColor(curr_frame,cv2.COLOR_BGR2RGB)
            #timestamp = timedelta(milliseconds = cap.get(cv2.CAP_PROP_POS_MSEC))
            #results.append([str(frame_no),str(timestamp)])
            #print("for frame : " + str(frame_no) + "   timestamp is: ", str(timestamp))
    else:
        break
    frame_no += 1

endtime = time.time()
exec_time = str(endtime - starttime)
print(f'total execution time = {exec_time} seconds')

cap.release()
cv2.destroyAllWindows()

In [ ]:
print(results)

In [ ]:
#2614-2637 01:47

### Capturing one frame of video

In [ ]:
frame_number = 1
cap = cv2.VideoCapture(video_file)
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
frame_exists, frame = cap.read()

if frame_exists:
    #plt.imshow(frame)
    face_object = get_faces_from_img(frame)
    embeddings = get_embedding_for_img(face_object[0][0][:,:,::-1])
    print(embeddings)
    plt.imshow(face_object[0][0][:,:,::-1])
    print(len(face_object))
    

cap.release()
cv2.destroyAllWindows()

In [ ]:
frame = get_frame(video_file,2639)
print('shape is', frame.shape)
print('pixel at (60,21)',frame[60,21,:])
print('pixel at (120,10)',frame[120,10,:])
plt.imshow(frame)

In [ ]:
fix_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
print('pixel at (120,10)',fix_frame[120,10,:])
plt.imshow(fix_frame)